## Homework5

In this homework, we will use Bank credit scoring dataset from [here](https://www.kaggle.com/datasets/kapturovalexander/bank-credit-scoring/data).

> **Note**: sometimes your answer doesn't match one of the options exactly. That's fine. 
Select the option that's closest to your solution.

> **Note**: we recommend using python 3.10 in this homework.

In [2]:
!python --version

Python 3.10.12


## Question 1

* Install Pipenv
* What's the version of pipenv you installed?
* Use `--version` to find out

In [5]:
!pipenv --version

pipenv, version 2023.10.3


## Question 2

* Use Pipenv to install Scikit-Learn version 1.3.1
* What's the first hash for scikit-learn you get in Pipfile.lock?

> **Note**: you should create an empty folder for homework
and do it there. 

In [7]:
# !pipenv install scikit-learn==1.3.1

Installing scikit-learn==1.3.1...
Resolving scikit-learn==1.3.1...
Added scikit-learn to Pipfile's [packages] ...
✔ Installation Succeeded-learn...
⠋ Installing scikit-learn...
Pipfile.lock not found, creating...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! Locking...
⠙ Locking...
Locking [dev-packages] dependencies...
Updated Pipfile.lock (0e0fec5cb0e411bbb2c1c4f81b061609272a25d0c1f780d06dd30aff281bed02)!
Installing dependencies from Pipfile.lock (1bed02)...
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.


        "hash": {
            "sha256": "0e0fec5cb0e411bbb2c1c4f81b061609272a25d0c1f780d06dd30aff281bed02"
        }